In [4]:
from StartingCandidateGenerator import StartingCandidateGenerator as SCG
from DataBaseInterface import DataBaseInterface as DBI
from CrossOperation import CrossOperation as CO
"Import necessary to fix atoms in the slab"
from ase.constraints import FixAtoms
from ase import Atoms
from ase.io import read
from generate_ceo2 import create_ceo2_111 as ceo2Slab
import numpy as np
from ase.visualize import view
import numpy as np
from ase.optimize import BFGS
from ase.calculators.emt import EMT

#--------------------------Define the fitness fucntion for your atoms-----------------------"

def fitness_function(atoms,env,reference = 0.0,au_energy = 0.0)-> float:

    at_num= atoms.get_atomic_numbers()
    pt_num = np.count_nonzero(at_num == 78)
    au_num= np.count_nonzero(at_num == 79)

    fre = atoms.get_potential_energy() - reference - au_num * (au_energy) - au_num*env
    return fre

#---------------------------Generate static part of the system------------------------------"
slab = ceo2Slab(number_of_layers=1,repetitions = (5,5,1),vacuum=12.0)
a = 12.0
slab = Atoms(cell=[a,a,a],
             pbc=True)
c = FixAtoms(indices=np.arange(len(slab)))
slab.set_constraint(c)
#If slab atoms arent fixed they will be generated  for all structures, but will be relaxed!"

#---------------------------Generate constant part of the system------------------------------"
#Part of the system that can be relaxed but that does not vary in number over the duration of the search"
constant = Atoms('Pt1')
print(constant.get_scaled_positions())

#---------Generate variable part of the system-(Single atom types only)------------------"
#Part of the system that can be relaxed and that varyies in number over the duration of the search"
#
variable_type = Atoms('Au')
#Considered number of variable type atoms n the search"
variable_range = [1,2,3,4,5,7,8,9,10]

#---------------------------Define starting population--------------------------------"

candidateGenerator = SCG(slab,constant,variable_type,variable_range)
crossing = CO(slab,constant,variable_type,variable_range,stc_change_chance = 0.5)
db = DBI('databaseGA.db')
population = 25

starting_pop = candidateGenerator.get_starting_population(population_size=population)

#-----------------------------------Define Calculator----------------------------------------"

calc = EMT()

#Get Reference energies##########
constant.cell = slab.get_cell()
variable_type.cell = slab.get_cell()

constant.calc = calc
dyn = BFGS(constant)
dyn.run(steps=100, fmax=0.05)
ref = constant.get_potential_energy()
variable_type.calc = calc
dyn = BFGS(variable_type)
dyn.run(steps=100, fmax=0.05)
au_en = variable_type.get_potential_energy()
#####################################



for i in starting_pop:
    db.add_unrelaxed_candidate(i )

#---------------------------------Relax initial structures-----------------------------------"
env = -0.5 #Environment chem pot 
while db.get_number_of_unrelaxed_candidates() > 0:

    atoms = db.get_first_unrelaxed()
    
    atoms.calc = calc
    dyn = BFGS(atoms)
    dyn.run(steps=100, fmax=0.05)
    atoms.get_potential_energy()
    
    atoms.info['key_value_pairs']['raw_score'] = -fitness_function(atoms,env,reference = ref, au_energy = au_en)
   
    db.update_to_relaxed(atoms.info['key_value_pairs']['dbid'],atoms)

#--------------------------------------Find better stoich to srtart eval----------------------------
#Overall fitness value
steps = 10
sub_steps = 20
for i in range(steps):
    for j in range(sub_steps):
        atomslist = db.get_better_candidates(n=10)
        #Choose two of the most stable structures to pair
        cand1 = np.random.randint(0,10)
        cand2 = np.random.randint(0,10)
        while cand1 == cand2:
            cand2 = np.random.randint(0,10)
            
        #Mate the particles
        res  = crossing.cross(atomslist[cand1],atomslist[cand2])
        if(res is not None):
            if(len(atomslist[cand1])!= len(res) and len(atomslist[cand2]) != len(res)):
                print("--------------STCCHANGE-----------------")
            db.add_unrelaxed_candidate(res)
            
    
    while db.get_number_of_unrelaxed_candidates() > 0:

        atoms = db.get_first_unrelaxed()

        atoms.calc = calc
        dyn = BFGS(atoms)
        dyn.run(steps=100, fmax=0.05)
        atoms.get_potential_energy()

        atoms.info['key_value_pairs']['raw_score'] = -fitness_function(atoms,env,reference = ref, au_energy = au_en)

        db.update_to_relaxed(atoms.info['key_value_pairs']['dbid'],atoms)
        
atomslist = db.get_better_candidates(n=2)
for at in atomslist:
    print(at.numbers)
view(atomslist[0])

[[0. 0. 0.]]
[78 79]
[78 79]
[78 79 79]
[78 79 79]
[78 79 79 79]
[78 79 79 79]
[78 79 79 79 79]
[78 79 79 79 79]
[78 79 79 79 79 79]
[78 79 79 79 79 79]
[78 79 79 79 79 79 79 79]
[78 79 79 79 79 79 79 79]
[78 79 79 79 79 79 79 79 79]
[78 79 79 79 79 79 79 79 79]
[78 79 79 79 79 79 79 79 79 79]
[78 79 79 79 79 79 79 79 79 79]
[78 79 79 79 79 79 79 79 79 79 79]
[78 79 79 79 79 79 79 79 79 79 79]
      Step     Time          Energy         fmax
BFGS:    0 15:19:28        5.850000        0.0000
      Step     Time          Energy         fmax
BFGS:    0 15:19:28        3.800000        0.0000
      Step     Time          Energy         fmax
BFGS:    0 15:19:29        4.149172        5.3566
BFGS:    1 15:19:29        3.413986        3.9886
BFGS:    2 15:19:29        4.433464       14.2824
BFGS:    3 15:19:29        3.132328        2.3275
BFGS:    4 15:19:29        3.055124        1.1533
BFGS:    5 15:19:29        3.035136        0.2731
BFGS:    6 15:19:29        3.034113        0.0230
      

BFGS:   31 15:19:29        5.180436        0.5003
BFGS:   32 15:19:29        5.171051        0.2768
BFGS:   33 15:19:29        5.168043        0.1251
BFGS:   34 15:19:29        5.167238        0.0582
BFGS:   35 15:19:29        5.167045        0.0526
BFGS:   36 15:19:29        5.166574        0.0747
BFGS:   37 15:19:29        5.166092        0.0908
BFGS:   38 15:19:29        5.165476        0.0823
BFGS:   39 15:19:29        5.165088        0.0655
BFGS:   40 15:19:29        5.164817        0.0737
BFGS:   41 15:19:29        5.164471        0.0823
BFGS:   42 15:19:29        5.163510        0.1166
BFGS:   43 15:19:29        5.159629        0.2579
BFGS:   44 15:19:29        5.135779        0.6464
BFGS:   45 15:19:29        5.030577        0.8422
BFGS:   46 15:19:29        4.852078        0.9570
BFGS:   47 15:19:29        5.344938        5.4276
BFGS:   48 15:19:29        4.826480        0.7725
BFGS:   49 15:19:29        4.819869        0.7431
BFGS:   50 15:19:29        4.806946        0.7724


BFGS:   69 15:19:30        8.713254        0.1938
BFGS:   70 15:19:30        8.692502        0.3955
BFGS:   71 15:19:30        8.673830        1.0590
BFGS:   72 15:19:30        8.662731        0.8545
BFGS:   73 15:19:30        8.635921        1.1372
BFGS:   74 15:19:30        8.580235        1.1975
BFGS:   75 15:19:30        8.462635        1.0633
BFGS:   76 15:19:30        8.336612        0.8690
BFGS:   77 15:19:30        8.385743        1.6219
BFGS:   78 15:19:30        8.305147        0.6921
BFGS:   79 15:19:30        8.293472        0.7717
BFGS:   80 15:19:30        8.273528        0.7183
BFGS:   81 15:19:30        8.256284        0.6120
BFGS:   82 15:19:30        8.230176        0.7019
BFGS:   83 15:19:30        8.130235        1.1678
BFGS:   84 15:19:30        8.080549        1.3139
BFGS:   85 15:19:30        8.012959        0.8238
BFGS:   86 15:19:30        7.965582        0.7058
BFGS:   87 15:19:30        7.918680        0.7011
BFGS:   88 15:19:30        7.908858        1.0638


BFGS:   29 15:19:30        9.936386        1.1190
BFGS:   30 15:19:30        9.847712        1.2384
BFGS:   31 15:19:30       10.131654        4.4641
BFGS:   32 15:19:30        9.844151        1.5057
BFGS:   33 15:19:30        9.857500        1.4494
BFGS:   34 15:19:30        9.720535        1.0650
BFGS:   35 15:19:30        9.659690        1.0272
BFGS:   36 15:19:30        9.566031        1.0643
BFGS:   37 15:19:30        9.550009        2.1542
BFGS:   38 15:19:30        9.382426        0.6689
BFGS:   39 15:19:30        9.337666        0.5824
BFGS:   40 15:19:30        9.306586        0.5104
BFGS:   41 15:19:30        9.296439        0.3969
BFGS:   42 15:19:30        9.281280        0.2553
BFGS:   43 15:19:30        9.270860        0.2290
BFGS:   44 15:19:30        9.262040        0.1789
BFGS:   45 15:19:30        9.258817        0.1233
BFGS:   46 15:19:30        9.257769        0.0718
BFGS:   47 15:19:30        9.257160        0.0718
BFGS:   48 15:19:30        9.256705        0.0825


BFGS:   91 15:19:31        6.701110        0.1903
BFGS:   92 15:19:31        6.698806        0.1883
BFGS:   93 15:19:31        6.696406        0.1880
BFGS:   94 15:19:31        6.693260        0.1754
BFGS:   95 15:19:31        6.689515        0.1839
BFGS:   96 15:19:31        6.683476        0.1784
BFGS:   97 15:19:31        6.676313        0.1451
BFGS:   98 15:19:31        6.673450        0.0815
BFGS:   99 15:19:31        6.672694        0.0344
      Step     Time          Energy         fmax
BFGS:    0 15:19:31       20.592158       20.5811
BFGS:    1 15:19:31       14.439677        2.7115
BFGS:    2 15:19:31       14.173585        1.8678
BFGS:    3 15:19:31       13.682596        2.1583
BFGS:    4 15:19:31       12.919724        2.9845
BFGS:    5 15:19:31       12.199946        5.0973
BFGS:    6 15:19:31       11.853501        7.3388
BFGS:    7 15:19:31       11.294953        3.5395
BFGS:    8 15:19:31       10.780240        2.3247
BFGS:    9 15:19:31       10.503169        2.0950
B

...

BFGS:   57 15:20:11        6.892858        0.1161
BFGS:   58 15:20:11        6.890763        0.1027
BFGS:   59 15:20:11        6.888307        0.1297
BFGS:   60 15:20:11        6.885253        0.1560
BFGS:   61 15:20:11        6.881709        0.1669
BFGS:   62 15:20:11        6.877952        0.2029
BFGS:   63 15:20:11        6.874182        0.1902
BFGS:   64 15:20:11        6.869618        0.1970
BFGS:   65 15:20:11        6.863056        0.2757
BFGS:   66 15:20:11        6.852490        0.3506
BFGS:   67 15:20:11        6.836589        0.4518
BFGS:   68 15:20:11        6.817969        0.3663
BFGS:   69 15:20:11        6.799945        0.3609
BFGS:   70 15:20:11        6.784462        0.3659
BFGS:   71 15:20:11        6.769916        0.4256
BFGS:   72 15:20:11        6.755597        0.3927
BFGS:   73 15:20:11        6.761465        0.4202
BFGS:   74 15:20:11        6.740594        0.2133
BFGS:   75 15:20:11        6.734344        0.1849
BFGS:   76 15:20:11        6.729224        0.1651


BFGS:   28 15:20:12        9.820202        0.4644
BFGS:   29 15:20:12        9.789485        0.5701
BFGS:   30 15:20:12        9.756010        0.5732
BFGS:   31 15:20:12        9.705153        0.6013
BFGS:   32 15:20:12        9.622667        1.1320
BFGS:   33 15:20:12        9.497766        1.4681
BFGS:   34 15:20:12        9.364470        1.9615
BFGS:   35 15:20:12        9.006508        2.0034
BFGS:   36 15:20:12        8.646244        2.4322
BFGS:   37 15:20:12        8.118694        2.1067
BFGS:   38 15:20:12        7.916729        1.3520
BFGS:   39 15:20:12        7.762585        1.0714
BFGS:   40 15:20:12        7.584071        0.9387
BFGS:   41 15:20:12        7.440061        1.1794
BFGS:   42 15:20:12        7.265172        1.1263
BFGS:   43 15:20:12        7.134868        0.7737
BFGS:   44 15:20:12        7.074871        0.6778
BFGS:   45 15:20:12        7.019805        0.6008
BFGS:   46 15:20:12        6.969280        0.4520
BFGS:   47 15:20:12        6.927410        0.3976


BFGS:   18 15:20:17        7.631814        1.0623
BFGS:   19 15:20:17        7.605336        0.7783
BFGS:   20 15:20:17        7.553531        0.4862
BFGS:   21 15:20:17        7.537452        0.3238
BFGS:   22 15:20:17        7.529164        0.2993
BFGS:   23 15:20:17        7.509367        0.3034
BFGS:   24 15:20:17        7.505217        0.2432
BFGS:   25 15:20:17        7.499790        0.1416
BFGS:   26 15:20:17        7.497953        0.1651
BFGS:   27 15:20:17        7.493016        0.2017
BFGS:   28 15:20:17        7.488417        0.1938
BFGS:   29 15:20:17        7.482604        0.2202
BFGS:   30 15:20:17        7.478224        0.1900
BFGS:   31 15:20:17        7.475585        0.1497
BFGS:   32 15:20:17        7.473486        0.1773
BFGS:   33 15:20:17        7.468298        0.2764
BFGS:   34 15:20:17        7.433931        0.6781
BFGS:   35 15:20:17        7.391571        1.0088
BFGS:   36 15:20:17        7.368444        0.5356
BFGS:   37 15:20:17        7.339589        0.3431


BFGS:   80 15:20:18        5.404387        0.3761
BFGS:   81 15:20:18        5.403463        0.3731
BFGS:   82 15:20:18        5.402177        0.3754
BFGS:   83 15:20:18        5.399766        0.3789
BFGS:   84 15:20:18        5.395996        0.3912
BFGS:   85 15:20:18        5.391129        0.4113
BFGS:   86 15:20:18        5.385920        0.4346
BFGS:   87 15:20:18        5.381765        0.4548
BFGS:   88 15:20:18        5.378420        0.4816
BFGS:   89 15:20:18        5.373244        0.5190
BFGS:   90 15:20:18        5.364873        0.5650
BFGS:   91 15:20:18        5.349683        0.6223
BFGS:   92 15:20:18        5.330626        0.6490
BFGS:   93 15:20:18        5.310458        0.5056
BFGS:   94 15:20:18        5.305178        0.4296
BFGS:   95 15:20:18        5.296063        0.4284
BFGS:   96 15:20:18        5.293808        0.4384
BFGS:   97 15:20:18        5.293171        0.4511
BFGS:   98 15:20:18        5.292743        0.4494
BFGS:   99 15:20:18        5.291818        0.4508


<Popen: returncode: None args: ['/home/jon/Documents/GrandCanonicalGA/ase_en...>